In [5]:
import sys
sys.path.insert(1, "../..")

from utils import llm_api
import numpy as np
import random
import string
import pandas as pd
from tasks.graph import graph_utils
import json
import re
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from utils import bipartite

The config for running the Relevance Estimation algorithms

In [2]:
config = {
    "model": "llama3.1:8b",
    "god_model": "gpt-3.5-turbo",
    "graph": {
        "samples": 5,
        "nodes": 30,
        "edges": 200,
        "vertex_step_size": 5
    },
    "algorithms": {
        "naive": {
            "chunk_size": 4
        },
        "bi_graph": {
            "shuffles": 4,
            "batch_size": 8
        }
    }
}

Generate the prompt given the task (I, q)

In [3]:
def generate_prompt(edges, vertex):
    prompt = "Consider an undirected graph with the following edges as pairs of source and target nodes: \n"

    for e in edges:
        prompt += str(e) + "\n"
    # prompt += str(edges)

    prompt += f"What is the degree of node {vertex}? Answer with just a number without further explanation."
    return prompt

## Bipartite

This part implements the bipartite evaluation method. Most of other implementations are in 'utils' dirs.

In [4]:
def build_prompt_for_scores(query, input_batch):
    prompt = "Consider a graph with the following list of edges:\n"
    for b in input_batch:
        prompt += f"- {b}\n"
    prompt += f"\nGive a list of comma-separated numbers as the relevance scores of each edge to the query '{query}'. The list should have the same number of elements as the edge list. Each number in the list corresponds to how much the edge is relevant to the query. Each number is an integer between 0 and 10. Answer in one line containing the comma separated scores."
    return prompt

def ask_score(input_batch, query):
    prompt = build_prompt_for_scores(query, input_batch)
    answer = llm_api.ask([prompt], config["model"])
    p1 = r"(\s*\d+\s*)"
    p2 = r"(,(\s*\d+\s*))+"
    pattern = "(" + p1 + (len(input_batch) - 1) * p2 + ")"
    for line in answer.split("\n"):
        if len(re.findall(pattern, line)) > 0:
            answer = re.findall(pattern, line)[0][0]
            print("[BI_GRAPH]", line, input_batch, query)
            break
    return [llm_api.take_out_number(a) for a in answer.split(",")]

Run the bipartite method given the hyper-parameters

In [ ]:
k = config["algorithms"]["bi_graph"]["shuffles"]
m = config["algorithms"]["bi_graph"]["batch_size"]

def run_bi_graph(prompt_graph, result_dict, node_size, step_size):
    prompt = graph_utils.get_list_of_edges(prompt_graph)
    random.shuffle(prompt) # Random Shuffle the list of edges

    for j in np.arange(node_size, step=step_size):        
        time.sleep(0.5)

        query = f"What is the degree of node {j}?"
        g, element_nodes = bipartite.create_bi_graph(k=k, m=m, prompt=prompt, query=query, ask_score=ask_score)
        bipartite.learn_bi_graph(g)
        new_prompt = sorted(prompt, key=lambda x: element_nodes[x].payload, reverse=True)

        result_dict["vertex"].append(j)
        result_dict["original_prompt"].append(prompt)
        result_dict["generated_prompt"].append(new_prompt)
        result_dict["algorithm"].append("bi_graph")
        result_dict["model"].append(config["model"])
        result_dict["edge_size"].append(config["graph"]["edges"])

## Warmup

This is the warm-up algorithm implementation which is comparably easier than the previous one.

In [ ]:
def get_related_edges(graph, question, model="gpt-3.5-turbo"):
    ans = llm_api.ask(f"Consider an undirected graph with the following edges as pairs of source and target vertecies: \n{graph}\n Which edges are directly related to the query '{question}'? Just give a list of tuples with no furthur explanation.", model)
    return ans

# chunk_size = 4
chunk_size = config["algorithms"]["naive"]["chunk_size"]

def run_naive(g, result_dict, node_size, step_size):
    edges = graph_utils.get_list_of_edges(g)
    random.shuffle(edges)
    chs = []
    for h in range(len(edges) // chunk_size):
        chs.append(edges[h * chunk_size: (h + 1) * chunk_size])

    for j in np.arange(node_size, step=step_size):
        time.sleep(0.5)

        question = f"What is the degree of vertex {j}?"
        reg = r'(\(\d+,( |)\d+\))'

        related = []
        not_related = []

        for ch in chs:
            time.sleep(1)
            result = get_related_edges(ch, question, config["model"])
            print("[Naive] related edges: ", result)
            eds = [eval(x[0]) for x in re.findall(reg, result)]
            not_eds = list(filter(lambda x: x not in eds, ch))
            related.extend(eds)
            not_related.extend(not_eds)

        final_edges = related + not_related
        final_edges = final_edges[:len(edges)]


        result_dict["vertex"].append(j)
        result_dict["original_prompt"].append(edges)
        result_dict["generated_prompt"].append(related + not_related)
        result_dict["algorithm"].append("warmup")
        result_dict["model"].append(config["model"])
        result_dict["edge_size"].append(config["graph"]["edges"])

## Opt & Random

Assessing the upper and lower bounds for each given task

In [ ]:
def sort_edges(edges, vertex):
    rels = list(filter(lambda x: vertex in x, edges))
    non_rels = list(filter(lambda x: vertex not in x, edges))
    return rels + non_rels

def find_optimal_prompt(g, result_dict, node_size, step_size):
    edges = graph_utils.get_list_of_edges(g)
    for j in np.arange(node_size, step=step_size):
        sorted_edges = sort_edges(edges, j)
        
        result_dict["vertex"].append(j)
        result_dict["original_prompt"].append(edges)
        result_dict["generated_prompt"].append(sorted_edges)
        result_dict["algorithm"].append("opt")
        result_dict["model"].append(config["model"])
        result_dict["edge_size"].append(config["graph"]["edges"])

In [ ]:
def run_random(g, result_dict, node_size, step_size):
    edges = graph_utils.get_list_of_edges(g)
    random.shuffle(edges)
    for j in np.arange(node_size, step=step_size):
        time.sleep(0.5)
        
        result_dict["vertex"].append(j)
        result_dict["original_prompt"].append(edges)
        result_dict["generated_prompt"].append(edges)
        result_dict["algorithm"].append("random")
        result_dict["model"].append(config["model"])
        result_dict["edge_size"].append(config["graph"]["edges"])